# BASIC CRUD OPERATIONS

In [15]:
import pandas as pd
from datetime import datetime
from pymongo import MongoClient
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

In [16]:
print("Collections in eduhub:")
print(db.list_collection_names())

Collections in eduhub:
['assignments', 'enrollments', 'submissions', 'users', 'lessons', 'courses']


## CREATE OPERATIONS

In [17]:
# Adding a new student
new_student = {
    "userId": "STU020",
    "email": "student17@example.com",
    "firstName": "Ada",
    "lastName": "Eze",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
        "bio": "Passionate learner exploring data science and analytics.",
        "avatar": "https://example.com/avatar16.png",
        "skills": ["Python", "SQL"]
    },
    "isActive": True
}

try:
    db.users.insert_one(new_student)
    print("Added new student")
except Exception as e:
    print("Error adding new student:", e)


# Creating a new course
new_course = {
    "courseId": "CRS012",
    "title": "Data Visualization with Power BI",
    "description": "Learn to design and develop interactive dashboards with Power BI.",
    "instructorId": "INST001",
    "category": "Data Science",
    "level": "beginner",
    "duration": 40,
    "price": 99.99,
    "tags": ["data", "visualization", "business-intelligence"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now(),
    "isPublished": True
}

try:
    db.courses.insert_one(new_course)
    print("Added new course")
except Exception as e:
    print("Error adding new course:", e)


# Enrolling a student in a course
new_enrollment = {
    "enrollmentId": "ENR019",
    "studentId": "STU016",
    "courseId": "CRS009",
    "enrolledAt": datetime.now(),
    "status": "active"
}

try:
    db.enrollments.insert_one(new_enrollment)
    print("Enrolled student in course")
except Exception as e:
    print("Error enrolling student:", e)


# Adding a new lesson to an existing course
new_lesson = {
    "lessonId": "LES030",
    "courseId": "CRS009",
    "title": "Getting Started with Power BI",
    "content": "This lesson introduces the Power BI interface and its core features.",
    "order": 1,
    "resources": ["https://docs.microsoft.com/en-us/power-bi/"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now()
}

try:
    db.lessons.insert_one(new_lesson)
    print("Added new lesson to course")
except Exception as e:
    print("Error adding new lesson:", e)

Added new student
Added new course
Enrolled student in course
Added new lesson to course


## READ OPERATIONS


### Active Students 

In [18]:
#Finding all active student
try:
    active_students = db.users.find(
        {"role": "student", "isActive": True},
        {"_id": 0, "userId": 1, "firstName": 1, "lastName": 1, "email": 1}
    )
    active_students_list = list(active_students)
    
    print("Active Students:")
    for student in active_students_list:
        print(student)
except Exception as e:
    print("Error fetching active students:", e)

Active Students:
{'userId': 'STU004', 'email': 'jorge23@example.org', 'firstName': 'Shane', 'lastName': 'Scott'}
{'userId': 'STU006', 'email': 'jgraham@example.org', 'firstName': 'Mackenzie', 'lastName': 'Richardson'}
{'userId': 'STU008', 'email': 'whiteemily@example.com', 'firstName': 'Amy', 'lastName': 'Alvarez'}
{'userId': 'STU010', 'email': 'amandahall@example.com', 'firstName': 'Robin', 'lastName': 'Barnett'}
{'userId': 'STU014', 'email': 'paul61@example.org', 'firstName': 'Kristin', 'lastName': 'Brown'}
{'userId': 'STU015', 'email': 'claudiaferguson@example.org', 'firstName': 'Michael', 'lastName': 'Haney'}
{'userId': 'STU020', 'email': 'student17@example.com', 'firstName': 'Ada', 'lastName': 'Eze'}


### Retrieving Course Details with Instructor Information

In [19]:
# Retreving course details with instructors information
try:
    course_with_instructor = db.courses.aggregate([
        {
            "$lookup": {
                "from": "users",
                "localField": "instructorId",
                "foreignField": "userId",
                "as": "instructor"
            }
        },
        { "$unwind": "$instructor" },  
        {
            "$project": {
                "_id": 0,
                "courseId": 1,
                "title": 1,
                "category": 1,
                "level": 1,
                "instructor.firstName": 1,
                "instructor.lastName": 1,
                "instructor.email": 1
            }
        }
    ])
    
    courses_list = list(course_with_instructor)
    
    print("Courses with Instructor Info:")
    for course in courses_list:
        print(course)
except Exception as e:
    print("Error fetching courses with instructor info:", e)

Courses with Instructor Info:
{'courseId': 'CRS001', 'title': 'Cloud Computing with AWS', 'category': 'Cloud Computing', 'level': 'beginner', 'instructor': {'email': 'rogerstrevor@example.com', 'firstName': 'Tammy', 'lastName': 'Rogers'}}
{'courseId': 'CRS002', 'title': 'Introduction to Data Science', 'category': 'Data Science', 'level': 'beginner', 'instructor': {'email': 'jonathan02@example.com', 'firstName': 'Mark', 'lastName': 'Carter'}}
{'courseId': 'CRS003', 'title': 'Frontend Development with React', 'category': 'Web Development', 'level': 'beginner', 'instructor': {'email': 'johnsondominique@example.com', 'firstName': 'Courtney', 'lastName': 'Mills'}}
{'courseId': 'CRS004', 'title': 'Full-Stack Web Development', 'category': 'Web Development', 'level': 'intermediate', 'instructor': {'email': 'npierce@example.org', 'firstName': 'Charles', 'lastName': 'Fox'}}
{'courseId': 'CRS005', 'title': 'DevOps Fundamentals', 'category': 'Cloud Computing', 'level': 'advanced', 'instructor': {'

###  Getting all Courses in a Specific Category

In [20]:
#Getting all courses in a specific category
category = "Data Science"

try:
    courses_in_category = db.courses.find(
        {"category": category},
        {"_id": 0, "courseId": 1, "title": 1, "level": 1}
    )
    
    print(f"Courses in {category}:")
    for course in courses_in_category:
        print(course)

except Exception as e:
    print(f"Error fetching courses in category '{category}': {e}")

Courses in Data Science:
{'courseId': 'CRS002', 'title': 'Introduction to Data Science', 'level': 'beginner'}
{'courseId': 'CRS007', 'title': 'Python for Data Analysis', 'level': 'advanced'}
{'courseId': 'CRS012', 'title': 'Data Visualization with Power BI', 'level': 'beginner'}


### Finding Students Enrolled in a Particular Course


In [21]:
# Finding student enrolled in a particular course
course_id = "CRS001"

try:
    students_in_course = db.enrollments.aggregate([
        { "$match": {"courseId": course_id} },
        {
            "$lookup": {
                "from": "users",
                "localField": "studentId",
                "foreignField": "userId",
                "as": "student"
            }
        },
        { "$unwind": "$student" },
        {
            "$project": {
                "_id": 0,
                "student.userId": 1,
                "student.firstName": 1,
                "student.lastName": 1,
                "student.email": 1
            }
        }
    ])

    print(f"Students enrolled in {course_id}:")
    for student in students_in_course:
        print(student)

except Exception as e:
    print(f"Error fetching students for course '{course_id}': {e}")

Students enrolled in CRS001:
{'student': {'userId': 'STU005', 'email': 'stephensrobert@example.org', 'firstName': 'Elaine', 'lastName': 'Maldonado'}}


### Searching Courses by Title (case-insensitive, partial match)

In [22]:
# Searching courses by Title
search_term = ".*python.*"

try:
    matched_courses = db.courses.find(
        {"title": {"$regex": search_term, "$options": "i"}},
        {"_id": 0, "courseId": 1, "title": 1, "category": 1, "level": 1}
    )

    print(f"Courses matching '{search_term}':")
    for course in matched_courses:
        print(course)

except Exception as e:
    print(f"Error searching courses with term '{search_term}': {e}")

Courses matching '.*python.*':
{'courseId': 'CRS007', 'title': 'Python for Data Analysis', 'category': 'Data Science', 'level': 'advanced'}


## UPDATE OPERATIONS

### Update a User’s Profile Information

In [23]:
#Updating a Useers Profile information
user_id = "STU005"

try:
    db.users.update_one(
        {"userId": user_id},
        {
            "$set": {
                "profile.bio": "Enthusiastic learner with interest in AI and Machine Learning.",
                "profile.avatar": "https://example.com/new_avatar.png",
                "profile.skills": ["Python", "SQL", "Machine Learning"]
            }
        }
    )
    print(f"User {user_id} profile updated successfully.")

except Exception as e:
    print(f"Error updating user {user_id}: {e}")

User STU005 profile updated successfully.


### Mark a Course as Published

In [25]:
course_id = "CRS003"

try:
    db.courses.update_one(
        {"courseId": course_id},
        {"$set": {"isPublished": True, "updatedAt": datetime.now()}}
    )
    print(f"Course {course_id} marked as published successfully.")

except Exception as e:
    print(f"Error marking course {course_id} as published: {e}")

Course CRS003 marked as published successfully.


### Update Assignment Grades

In [26]:
#Updating Assignment Grades
submission_id = "SUB010"

try:
    db.submissions.update_one(
        {"submissionId": submission_id},
        {
            "$set": {
                "grade": 80,
                "feedback": "Excellent work, keep it up!",
                "gradedAt": datetime.now()
            }
        }
    )
    print(f"Updated grade for submission {submission_id} successfully.")

except Exception as e:
    print(f"Error updating grade for submission {submission_id}: {e}")

Updated grade for submission SUB010 successfully.


### Add Tags to an Existing Course

In [27]:
#Adding Tags to an Existing Course
course_id = "CRS005"

try:
    db.courses.update_one(
        {"courseId": course_id},
        {"$addToSet": {"tags": {"$each": ["AI", "Data Science", "Python"]}}}
    )
    print(f"Tags updated for course {course_id} successfully.")

except Exception as e:
    print(f"Error updating tags for course {course_id}: {e}")

Tags updated for course CRS005 successfully.


## DELETE OPERATIONS

### Remove a User (soft delete by setting isActive to false)


In [29]:
# Removing user
user_id = "STU006"

try:
    db.users.update_one(
        {"userId": user_id},
        {"$set": {"isActive": False, "updatedAt": datetime.now()}}
    )
    print(f"User {user_id} has been deactivated successfully.")
except Exception as e:
    print(f"Error deactivating user {user_id}: {e}")


User STU006 has been deactivated successfully.


### Deleting  an Enrollment

In [30]:
#Deleting an Enrollment
enrollment_id = "ENR005"

try:
    delete_result = db.enrollments.delete_one({"enrollmentId": enrollment_id})
    print(f"Deleted {delete_result.deleted_count} enrollment(s) with ID {enrollment_id}")
except Exception as e:
    print(f"Error deleting enrollment {enrollment_id}: {e}")

Deleted 1 enrollment(s) with ID ENR005


###  Remove a Lesson from a Course

In [31]:
# Removing a lesson from a Course
lesson_id = "LES008"

try:
    delete_result = db.lessons.delete_one({"lessonId": lesson_id})
    print(f"Deleted {delete_result.deleted_count} lesson(s) with ID {lesson_id}")
except Exception as e:
    print(f"Error deleting lesson {lesson_id}: {e}")

Deleted 1 lesson(s) with ID LES008
